In [9]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [30]:
import pandas as pd

# Contoh path dataset
df = pd.read_csv('/content/drive/MyDrive/RECOMENDATION RECIPES/reseptrainingg.csv')

# Tampilkan 5 baris pertama
df.head()


,Title,Ingredients,Steps
0,Lumpia udang kulit tahu,"""['50 gram ayam potong kotak kecil', '200 gram...",Campur ayam & udang dengan semua bumbu & daun ...
1,Bakso Ayam Udang,"""['400 gr ayam giling', '250 gr udang kupas', ...","Campur semua bahan dalam food processor, kecua..."
2,Udang ala pop corn,"""['1/4 kg udang basah ukuran sedang', '1 bungk...",Buang kepala dan cangkang udang.--Cuci bersih ...
3,Bakwan Udang Sederhana,"""['1/2 kg Tepung terigu (sudah di ayak)', '1/2...","Haluskan bawang putih, garam, picin dan merica..."
4,Udang Goreng Tepung Mayonaise,"""['300 gr udang (buang kepalanya)', 'Jeruk nip...","Cuci bersih udang, tambahkan jeruk nipis supay..."


In [11]:
df.isnull().sum()


,0
Title,0
Ingredients,0
Steps,0


In [19]:
# Hapus tanda kurung siku dan spasi berlebih, lalu split pakai koma
def clean_ingredient_string(s):
    # Hapus tanda [ ], \, "
    s = s.replace('[', '').replace(']', '').replace('\\', '').replace('"', '').replace("'", "")
    # Pisahkan jadi list
    items = [i.strip() for i in s.split(',')]
    return items

# Terapkan ke dataframe
df['Ingredient List'] = df['Ingredients'].apply(clean_ingredient_string)

# Cek hasil
df['Ingredient List'][0]


['50 gram ayam potong kotak kecil',
 '200 gram udang besar potong kecil',
 '1 lembar daun bawang iris halus',
 '1/2 sendok teh garam halus/sesuai selera',
 '1/2 sendok teh gula',
 '1/4 sendok teh merica bubuk',
 '1 sendok teh kecap ikan',
 '1 sendok teh saus raja rasa',
 '1 sendok teh saus tiram',
 '1 sendok teh minyak wijen',
 '1 sendok makan tepung sagu',
 '1 sendok teh tepung terigu',
 '1 butir telur ambil bagian putihnya saja',
 'kocok lepas',
 '2 lembar kembang tahu kering / kulit tahu direndam sebentar']

In [21]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

In [20]:
import pandas as pd
import re
from sklearn.neighbors import NearestNeighbors
import numpy as np

# Fungsi ekstrak bahan dan gramasi
def parse_ingredients(ingredient_list):
    ingredient_dict = {}
    for item in ingredient_list:
        # Contoh parsing sederhana: angka + nama bahan
        match = re.match(r"([\d/.]+)\s*(gram|gr|kg|butir|sendok)?\s*(.*)", item.lower())
        if match:
            qty = eval(match.group(1).replace(',', '.'))  # handle angka 1/2 atau desimal
            unit = match.group(2) or ''
            name = match.group(3).strip()

            # Konversi satuan ke gram (basic)
            if 'kg' in unit:
                qty *= 1000
            elif 'butir' in unit:
                qty *= 50  # anggap 1 butir telur 50 gram (bisa kamu sesuaikan)
            # bisa tambah rules lain

            ingredient_dict[name] = qty
    return ingredient_dict


In [25]:
# Bersihkan list ingredients
# df['Ingredient List'] = df['Ingredients'].apply(eval)  # kalau awalnya string list ['...', '...'] # This line caused the error

# Ubah jadi dict bahan
df['Parsed'] = df['Ingredient List'].apply(parse_ingredients)

# Buat daftar semua bahan unik
all_ingredients = set()
for ing in df['Parsed']:
    all_ingredients.update(ing.keys())

# Buat DataFrame numerik
def ingredient_to_vector(parsed, all_ingredients):
    return [parsed.get(i, 0) for i in all_ingredients]

X_ingredients = np.array([ingredient_to_vector(row, all_ingredients) for row in df['Parsed']])

ingredient_columns = list(all_ingredients)
df_ingredients = pd.DataFrame(X_ingredients, columns=ingredient_columns)

In [26]:
knn = NearestNeighbors(n_neighbors=3, metric='euclidean')
knn.fit(df_ingredients.values)


NearestNeighbors(metric='euclidean', n_neighbors=3)

In [27]:
def calculate_portions(user_stock, recipe_stock):
    porsis = []
    for bahan, qty in recipe_stock.items():
        if bahan in user_stock and recipe_stock[bahan] > 0:
            porsis.append(user_stock[bahan] / recipe_stock[bahan])
        else:
            porsis.append(0)
    return round(min(porsis), 2)


In [28]:
def recommend_recipe(user_input):
    # Convert user input dict to vector
    user_vector = np.array([user_input.get(i, 0) for i in ingredient_columns]).reshape(1, -1)
    distances, indices = knn.kneighbors(user_vector)

    recommended_recipes = []
    for idx in indices[0]:
        recipe_title = df.iloc[idx]['Title']
        recipe_ingredients = df.iloc[idx]['Parsed']
        steps = df.iloc[idx]['Steps']

        # Hitung porsi sesuai bahan user
        max_porsi = calculate_portions(user_input, recipe_ingredients)

        recommended_recipes.append({
            'Title': recipe_title,
            'Max Porsi': max_porsi,
            'Ingredients': recipe_ingredients,
            'Steps': steps
        })
    return recommended_recipes


In [29]:
user_stock = {
    'ayam potong': 400,
    'tepung terigu': 100,
    'telur': 100,
}

hasil = recommend_recipe(user_stock)

for resep in hasil:
    print(f"Resep: {resep['Title']}")
    print(f"Porsi yang bisa dibuat: {resep['Max Porsi']}")
    print("Bahan:")
    for b, g in resep['Ingredients'].items():
        print(f"  {b}: {g} gram")
    print("Langkah-langkah:", resep['Steps'])
    print("="*50)


Resep: Dadar telor ikan tongkol
Porsi yang bisa dibuat: 0
Bahan:
  ikan tongkol: 1 gram
  telur: 2 gram
  ikat daun bawang: 1 gram
  sdt garam: 0.5 gram
Langkah-langkah: Goreng ikan tongkol setengah matang lalu suwirÂ²--PotongÂ² bawang daun lalu campur semua kocok jadi satu dalam wadah--Goreng sesendokÂ² klo dah kuning keemasan angkat lalu tiriskan, soap disajikan sebagai lauk sayur asem cucok banget--Bisa jg langsung digoreng jadi satu pas nanti dipotongÂ² sesuai selera, selamat mencoba moms ðŸ˜Š--
Resep: Semur Ayam Kentang Ala Mama Kirana
Porsi yang bisa dibuat: 0
Bahan:
  ekor ayam: 0.5 gram
Langkah-langkah: Cuci bersih ayam lalu baluri dengan perasan jeruk nipis--Cuci lagi ayam dan baluri dengan garam lalu rebus sebentar lalu di goreng--Kentang saya goreng bisa di sesuaikan sesuai selera masingÂ²--Tips goreng ayam agar hasil nya tdk keras : di goreng hanya sebentar--Tumis bawang bombay lalu masukan bumbu halus dan masukkan semua rempah + irisan tomat + kecap manis--Setelah bumbu ha